<a href="https://colab.research.google.com/github/vkjadon/tf/blob/main/tf100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let us try to create a neural networks to learn the following relationship between two numbers. We can write a simple function as below:

```
def hw_function(x):
    y = (2 * x) - 1
    return y
```

We can try the same by using neural network by feeding with a set of x's and y's, it should be able to figure out the relationship between them. 

In the neural network approach we can fit any relationship of the data and the function above follows a standard relation.

## Imports

Let's import [TensorFlow](https://www.tensorflow.org/) and call it `tf` as a general convention.

[`numpy`](https://numpy.org) helps to represent data as arrays and to optimize numerical operations.

In [1]:
import tensorflow as tf
print(tf.__version__)

2.9.2


## Define and Compile the Neural Network

We will create the simplest possible neural network with 1 layer with 1 neuron, and the input shape to it is just 1 value.    
We will build this model using Keras' [Sequential](https://keras.io/api/models/sequential/) class which allows us to define the network as a sequence of [layers](https://keras.io/api/layers/). We can use a single [Dense](https://keras.io/api/layers/core_layers/dense/) layer to build this simple network as shown below.

In [29]:
# Build a simple Sequential model
model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])

In [24]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 3)                 6         
                                                                 
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


Now, we need to compile the neural network. The compliation requires to specify 2 functions: a [loss](https://keras.io/api/losses/) and an [optimizer](https://keras.io/api/optimizers/).

When the computer try to 'learn' `y=2x-1`, it makes a guess and may assume the function be `y=10x+10`. The `loss` function measures the guessed answers against the known correct answers and measures how well or how badly it did.

It then uses the `optimizer` function to make another guess. Based on how the loss function went, it will try to minimize the loss. At that point maybe it will come up with something like `y=5x+5`, which, while still pretty bad, is closer to the correct result (i.e. the loss is lower).

It will repeat this for the number of _epochs_ which you will see shortly. But first, here's how you will tell it to use [mean squared error](https://keras.io/api/losses/regression_losses/#meansquarederror-function) for the loss and [stochastic gradient descent](https://keras.io/api/optimizers/sgd/) for the optimizer. You don't need to understand the math for these yet, but you can see that they work!

Over time, you will learn the different and appropriate loss and optimizer functions for different scenarios. 


In [30]:
# Compile the model
model.compile(optimizer='sgd', loss='mean_squared_error')

## Providing the Data

Next up, you will feed in some data. In this case, you are taking 6 X's and 6 Y's. You can see that the relationship between these is `y=2x-1`, so where `x = -1`, `y=-3` etc. 

The de facto standard way of declaring model inputs and outputs is to use `numpy`, a Python library that provides lots of array type data structures. You can specify these values by building numpy arrays with [`np.array()`](https://numpy.org/doc/stable/reference/generated/numpy.array.html).

In [31]:
import numpy as np
# Declare model inputs and outputs for training
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# Training the Neural Network

The process of training the neural network, where it 'learns' the relationship between the x's and y's is in the [`model.fit()`](https://keras.io/api/models/model_training_apis/#fit-method)  call. This is where it will go through the loop we spoke about above: making a guess, measuring how good or bad it is (aka the loss), using the opimizer to make another guess etc. It will do it for the number of `epochs` you specify. When you run this code, you'll see the loss on the right hand side.

In [34]:
# Train the model
model.fit(xs, ys, epochs=550)

Epoch 1/550
1/1 [==============================] - 0s 12ms/step - loss: 0.2852
Epoch 2/550
1/1 [==============================] - 0s 8ms/step - loss: 0.2793
Epoch 3/550
1/1 [==============================] - 0s 8ms/step - loss: 0.2736
Epoch 4/550
1/1 [==============================] - 0s 11ms/step - loss: 0.2680
Epoch 5/550
1/1 [==============================] - 0s 5ms/step - loss: 0.2625
Epoch 6/550
1/1 [==============================] - 0s 8ms/step - loss: 0.2571
Epoch 7/550
1/1 [==============================] - 0s 7ms/step - loss: 0.2518
Epoch 8/550
1/1 [==============================] - 0s 7ms/step - loss: 0.2466
Epoch 9/550
1/1 [==============================] - 0s 6ms/step - loss: 0.2416
Epoch 10/550
1/1 [==============================] - 0s 6ms/step - loss: 0.2366
Epoch 11/550
1/1 [==============================] - 0s 7ms/step - loss: 0.2317
Epoch 12/550
1/1 [==============================] - 0s 6ms/step - loss: 0.2270
Epoch 13/550
1/1 [==============================] - 0s 7ms/

Ok, now you have a model that has been trained to learn the relationship between `x` and `y`. You can use the [`model.predict()`](https://keras.io/api/models/model_training_apis/#predict-method) method to have it figure out the `y` for a previously unknown `x`. So, for example, if `x=10`, what do you think `y` will be? Take a guess before you run this code:

In [35]:
# Make a prediction
print(model.predict([10.0]))

1/1 [==============================] - 0s 28ms/step
[[18.994774]]


You might have thought `19`, right? But it ended up being a little under. Why do you think that is? 

Remember that neural networks deal with probabilities. So given the data that we fed the model with, it calculated that there is a very high probability that the relationship between `x` and `y` is `y=2x-1`, but with only 6 data points we can't know for sure. As a result, the result for 10 is very close to 19, but not necessarily 19.

As you work with neural networks, you'll see this pattern recurring. You will almost always deal with probabilities, not certainties, and will do a little bit of coding to figure out what the result is based on the probabilities, particularly when it comes to classification.


###Multiple Layers

In [38]:
#We can create a Sequential model by passing a list of layers to the Sequential constructor:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(2, input_shape=(6,), activation="relu"),
        tf.keras.layers.Dense(3, activation="relu"),
        tf.keras.layers.Dense(4),
    ]
)
print(len(model.layers))

3


In [43]:
#We can create a Sequential model incrementally via the add() method:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(2, activation="relu"))
model.add(tf.keras.layers.Dense(3, activation="relu"))
model.add(tf.keras.layers.Dense(4))
print(len(model.layers))

# Call the model on a test input
x = tf.ones((2, 4))
print("Input :", x)
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

3
Input : tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(2, 4), dtype=float32)
Number of weights after calling the model: 6


In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (2, 2)                    10        
                                                                 
 dense_5 (Dense)             (2, 3)                    9         
                                                                 
 dense_6 (Dense)             (2, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


However, it can be very useful when building a Sequential model incrementally to be able to display the summary of the model so far, including the current output shape. In this case, you should start your model by passing an Input object to your model, so that it knows its input shape from the start:

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(4,)))
model.add(tf.keras.layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


A simple alternative is to just pass an input_shape argument to your first layer:


In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(2, activation="relu", input_shape=(4,)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Models built with a predefined input shape like this always have weights (even before seeing any data) and always have a defined output shape.

In general, it's a recommended best practice to always specify the input shape of a Sequential model in advance if you know what it is.

In [13]:
# Optionally, the first layer can receive an `input_shape` argument:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(16,)))
# Afterwards, we do automatic shape inference:
model.add(tf.keras.layers.Dense(4))

In [14]:
# This is identical to the following:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(16,)))
model.add(tf.keras.layers.Dense(8))


In [15]:
# Note that you can also omit the `input_shape` argument.
# In that case the model doesn't have any weights until the first call
# to a training/evaluation method (since it isn't yet built):
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
# model.weights not created yet


In [16]:
# Whereas if you specify the input shape, the model gets built
# continuously as you are adding layers:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(16,)))
model.add(tf.keras.layers.Dense(4))
len(model.weights)
# Returns "4"


4

In [17]:
# When using the delayed-build pattern (no input shape specified), you can
# choose to manually build your model by calling
# `build(batch_input_shape)`:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(4))
model.build((None, 16))
len(model.weights)
# Returns "4"

4

In [18]:
# Note that when using the delayed-build pattern (no input shape specified),
# the model gets built the first time you call `fit`, `eval`, or `predict`,
# or the first time you call the model on some input data.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
model.fit(x, y, batch_size=32, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 379ms/step - loss: 0.2187
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 0.1851
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.1661
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 0.1554
Epoch 5/10
1/1 [==============================] - 0s 9ms/step - loss: 0.1494
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 0.1460
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 0.1441
Epoch 8/10
1/1 [==============================] - 0s 14ms/step - loss: 0.1430
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.1424
Epoch 10/10
1/1 [==============================] - 0s 11ms/step - loss: 0.1420
